In [1]:
import numpy as np
import pandas as pd
import math
import sys, os
#import seaborn as sns
#from scipy import stats
#from pathlib import Path
#import matplotlib.pyplot as plt
#from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
#from sklearn.metrics import mean_squared_error
#from sklearn.metrics import mean_squared_error
#from sklearn.preprocessing import MinMaxScaler
#from sklearn.datasets import fetch_california_housing
#import math
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
import optuna
from optuna.samplers import TPESampler
#import lightgbm as lbgm
#from xgboost import XGBRegressor
#import xgboost as xgb
from catboost import CatBoostRegressor
#from lightgbm.sklearn import lbgmRegressor
#from sklearn.ensemble import RandomForestRegressor
import pickle
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

In [2]:
train_time = 1 * 60 * 60
objective = 'catregressor'

In [3]:
sys.path.append(os.path.realpath('..'))

In [4]:
train_df = pd.read_csv(r"..\data\train.csv", index_col=0)
test_df = pd.read_csv(r"..\data\test.csv", index_col=0)
sample_sub = pd.read_csv(r"..\data\sample_submission.csv", index_col=0)

In [5]:
del train_df['Over18']
del train_df['EmployeeCount']
del train_df['StandardHours']

In [6]:
columns_to_vectorize = ['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'OverTime']
for vector_target in columns_to_vectorize:
    print(vector_target)
    vectorizer = CountVectorizer()
    vectorizer.fit_transform(train_df[vector_target])
    train_df[f'{vector_target}_v'] = vectorizer.transform(train_df[vector_target]).toarray().argmax(axis=1)[:,None]
    vectorizer.fit_transform(test_df[vector_target])
    test_df[f'{vector_target}_v'] = vectorizer.transform(test_df[vector_target]).toarray().argmax(axis=1)[:,None]

BusinessTravel
Department
EducationField
Gender
JobRole
MaritalStatus
OverTime


In [7]:
features = ['Age', 'DailyRate', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction','HourlyRate', 'JobInvolvement', 'JobLevel', 'JobSatisfaction', 
       'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel', 'TotalWorkingYears',
       'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager',
        'BusinessTravel_v', 'Department_v', 'EducationField_v','Gender_v', 'JobRole_v', 'MaritalStatus_v', 'OverTime_v']
target = ['Attrition']

In [8]:
scaler = MinMaxScaler().fit(train_df[features])
X = scaler.transform(train_df[features])
X_test = scaler.transform(test_df[features])

scaler = MinMaxScaler().fit(train_df[target])
Y = scaler.transform(train_df[target])

In [9]:
RANDOM_STATE = 12 
FOLDS = 5
param_grid_history = {}

def render_model(param_grid):
    reg = CatBoostRegressor(iterations=20000,
                            **param_grid,
                            bootstrap_type='Bernoulli',
                            grow_policy='SymmetricTree',
                            #loss_function='Logloss',
                            eval_metric='AUC',
                            task_type="GPU",
                            random_state=1,)
    return reg
    
def train_model(reg, X_train, y_train, X_valid, y_valid):
    reg.fit(X_train, y_train, eval_set=(X_valid, y_valid), early_stopping_rounds=100, verbose=1000)

    preds = reg.predict(train_df[features])
    fpr, tpr, _ = roc_curve(train_df[target], preds)
    roc_auc = auc(fpr, tpr)
    return reg, roc_auc

def objective_v2(trial):
    scores = []
    param_grid = {
        'depth': trial.suggest_int('depth', 3, 6),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.9),
        #'rsm': trial.suggest_float('rsm', 0.001, 0.9),
        #'subsample': trial.suggest_float('subsample', 0.1, 1),
        #'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 100), 
        #'one_hot_max_size': trial.suggest_int('one_hot_max_size', 1, 100), 
        #'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0.01, 20),
        'reg_lambda': trial.suggest_int('reg_lambda', 30, 100), 
        #'random_strength': trial.suggest_float('random_strength', 0.001, 0.9),
        #'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.001, 0.9),
    } 
        
    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=RANDOM_STATE)
    for fold, (train_idx, valid_idx) in enumerate(skf.split(X, Y)):
        print(10*"=", f"Fold={fold+1}", 10*"=")
        X_train, X_valid = train_df.iloc[train_idx][features], train_df.iloc[valid_idx][features]
        y_train , y_valid = train_df[target].iloc[train_idx] , train_df[target].iloc[valid_idx] 
        
        reg = render_model(param_grid)
        reg, roc_auc = train_model(reg, X_train, y_train, X_valid, y_valid)

        scores.append(roc_auc)
    mean_scores = np.mean(scores)
    param_grid_history[mean_scores] = param_grid

    return mean_scores

In [37]:
RANDOM_STATE = 12 
FOLDS = 5
param_grid_history = {}


def objective_v3(trial):
    oof = np.zeros(len(X))
    scores = []
    param_grid = {
        'depth': trial.suggest_int('depth', 3, 8),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.9),
        #'rsm': trial.suggest_float('rsm', 0.001, 0.9),
        #'subsample': trial.suggest_float('subsample', 0.1, 1),
        #'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 100), 
        #'one_hot_max_size': trial.suggest_int('one_hot_max_size', 1, 100), 
        #'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0.01, 20),
        'reg_lambda': trial.suggest_int('reg_lambda', 1, 5), 
        #'random_strength': trial.suggest_float('random_strength', 0.001, 0.9),
        #'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.001, 0.9),
    } 
        
    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=RANDOM_STATE)
    for fold, (train_idx, valid_idx) in enumerate(skf.split(X, Y)):
        print(10*"=", f"Fold={fold+1}", 10*"=")
        X_train, X_valid = train_df.iloc[train_idx][features], train_df.iloc[valid_idx][features]
        y_train , y_valid = train_df[target].iloc[train_idx] , train_df[target].iloc[valid_idx] 
        
        reg = CatBoostRegressor(iterations=20000,
                            **param_grid,
                            bootstrap_type='Bernoulli',
                            grow_policy='SymmetricTree',
                            
                            #loss_function='Logloss',
                            eval_metric='AUC',
                            task_type="GPU",
                            random_state=1,)

        reg.fit(X_train, y_train, eval_set=(X_valid, y_valid), early_stopping_rounds=100, verbose=1000)
            
        oof[valid_idx] = reg.predict(X_valid)

    roc_auc = roc_auc_score(Y, oof)
    
    param_grid_history[roc_auc] = param_grid

    return roc_auc

In [38]:
study = optuna.create_study(direction='maximize', sampler=TPESampler(), study_name=objective)
study.optimize(objective_v3, timeout=train_time)

[I 2023-01-20 19:39:11,989] A new study created in memory with name: catregressor


========== Fold=1 ==========


Default metric period is 5 because AUC is/are not implemented for GPU
AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.7421030	best: 0.7421030 (0)	total: 8.84ms	remaining: 2m 56s
bestTest = 0.7970439189
bestIteration = 16
Shrink model to first 17 iterations.
========== Fold=2 ==========


Default metric period is 5 because AUC is/are not implemented for GPU
AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


...

0:	test: 0.7510135	best: 0.7510135 (0)	total: 8.24ms	remaining: 2m 44s
bestTest = 0.8381756757
bestIteration = 239
Shrink model to first 240 iterations.
========== Fold=2 ==========


Default metric period is 5 because AUC is/are not implemented for GPU
AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.8141470	best: 0.8141470 (0)	total: 8.85ms	remaining: 2m 57s


[W 2023-01-20 19:42:08,245] Trial 18 failed because of the following error: KeyboardInterrupt('')
Traceback (most recent call last):
  File "D:\source\repos\venv\Python310\lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\ryans\AppData\Local\Temp\ipykernel_8280\1324698494.py", line 38, in objective_v3
    reg.fit(X_train, y_train, eval_set=(X_valid, y_valid), early_stopping_rounds=100, verbose=1000)
  File "D:\source\repos\venv\Python310\lib\site-packages\catboost\core.py", line 5730, in fit
    return self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, baseline,
  File "D:\source\repos\venv\Python310\lib\site-packages\catboost\core.py", line 2355, in _fit
    self._train(
  File "D:\source\repos\venv\Python310\lib\site-packages\catboost\core.py", line 1759, in _train
    self._object._train(train_pool, test_pool, params, allow_clear_pool, init_model._object 

KeyboardInterrupt: 

In [ ]:
parm_grid_bytes = pickle.dumps(param_grid_history)
with open(f"rendered_data/{objective}_bytes.hex", "wb") as binary_file:
    binary_file.write(parm_grid_bytes)

In [39]:
param_grid_history = {}
with open(f"rendered_data/{objective}_bytes.hex", "rb") as input_file:
    param_grid_history = pickle.load(input_file)

In [40]:
def train(param_grid):
    reg = CatBoostRegressor(iterations=20000,
                        **param_grid,
                        bootstrap_type='Bernoulli',
                        grow_policy='SymmetricTree',
                        #loss_function='Logloss',
                        eval_metric='AUC',
                        task_type="GPU",
                        random_state=1,)
    
    X_train, X_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.20)
    
    reg.fit(X_train, y_train, eval_set=(X_valid, y_valid), early_stopping_rounds=100, verbose=1000)
    
    return reg

percent = math.ceil(len(param_grid_history.keys()) * .1)
top = sorted(list(param_grid_history.keys()))[-percent:]

train_preds = []
test_preds = []
for key in tqdm(top):
    model = train(param_grid_history[key])
    train_preds.append(model.predict(train_df[features]))
    test_preds.append(model.predict(test_df[features]))

train_final_preds = np.stack(train_preds).mean(0)
test_final_preds = np.stack(test_preds).mean(0)

  0%|                                                                                           | 0/40 [00:00<?, ?it/s]Default metric period is 5 because AUC is/are not implemented for GPU
AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.6717472	best: 0.6717472 (0)	total: 7.64ms	remaining: 2m 32s


  2%|██                                                                                 | 1/40 [00:02<01:35,  2.45s/it]

bestTest = 0.8087261395
bestIteration = 98
Shrink model to first 99 iterations.


Default metric period is 5 because AUC is/are not implemented for GPU
AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


...

bestTest = 0.8133223684
bestIteration = 21
Shrink model to first 22 iterations.


Default metric period is 5 because AUC is/are not implemented for GPU
AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.7606841	best: 0.7606841 (0)	total: 7.67ms	remaining: 2m 33s


100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [01:09<00:00,  1.74s/it]

bestTest = 0.8367398649
bestIteration = 57
Shrink model to first 58 iterations.


In [41]:
train_submission = pd.DataFrame(data={'id': train_df.index, target[0]: train_final_preds})
train_submission.to_csv(fr'rendered_data/{objective}_train_submission.csv', index=False)

test_submission = pd.DataFrame(data={'id': test_df.index, target[0]: test_final_preds})
test_submission.to_csv(fr'rendered_data/{objective}_test_submission.csv', index=False)